- 라이브러리 로드

In [1]:
import cv2
import numpy as np

In [2]:
import ipywidgets as widgets

In [3]:
import torch

In [4]:
import cmapy

- GPU 설정

In [5]:
gpu = 0

device = torch.device(f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)

- 모델 설정

In [6]:
model_path_list = {
    'handmotion': '../model_flask/models/handmotion/best.pt',
    'equip': '../model_flask/models/equip/best.pt',
}

- 모델 로드

In [7]:
models = {}

for model_name, model_path in model_path_list.items():
   models[model_name] = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True).to(device)
   
   if model_name != 'handmotion':
       models[model_name].conf = 0.5
       models[model_name].iou = 0.5

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\kimwo/.cache\torch\hub\master.zip
YOLOv5  2023-4-5 Python-3.10.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\kimwo/.cache\torch\hub\master.zip
YOLOv5  2023-4-5 Python-3.10.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


- 컬러맵

In [8]:
cmap = cmapy.cmap('nipy_spectral')

- background subtraction

In [9]:
fgbg = cv2.createBackgroundSubtractorMOG2()

- box color

In [10]:
box_color = {
    'OK': (0, 255, 0),
    'NO': (0, 0, 255),
    'HAND': (255, 0, 0)
}

def get_box_color(model, name):
    if model in ['equip']:
        if 'OK' in name:
            return box_color['OK']
        return box_color['NO']
    if model == 'handmotion':
        return box_color['HAND']
    return (255, 255, 255)

- 파일

In [11]:
files = ['KakaoTalk_20230405_161238950.mp4', 'KakaoTalk_20230405_161251064.mp4']

- 처리

In [12]:
for f in files:
    cap = cv2.VideoCapture(f)
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    out = cv2.VideoWriter(f'./output_{f}', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    
    mhi_history = np.zeros((height, width), dtype=np.int16)
    pos = 15
    
    while True:
        ret, frame = cap.read()
        if not ret:
            cap.release()
            cv2.destroyAllWindows()
            break
            
        cv2.waitKey(1)
            
        cv2.imshow('frame', frame)

        fgmask = fgbg.apply(frame)

        # make 24 frame of history based on fgmask
        mhi_history = np.where(
            fgmask == 255, 255 + 15, mhi_history)
        mhi_history = np.where(
            mhi_history > -100, mhi_history - pos, mhi_history)

        # make clip for safety
        mhi_history = np.clip(mhi_history, 0, 255)
        history_frame = mhi_history.astype(np.uint8)

        # color map
        color_map_history = cv2.applyColorMap(history_frame, cmap)
        
        results = []
        for model_name, model in models.items():
            if model_name == 'handmotion':
                result = model(color_map_history)
            else:
                result = model(frame)

            for i in range(len(result.xyxy[0])):
                r = result.pandas().xyxy[0].iloc[i, :].values.tolist()
                x1, y1, x2, y2, confidence, cls_id, cls_name = r
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                confidence, cls_id = float(confidence), int(cls_id)

                results.append({
                    'x1': x1,
                    'y1': y1,
                    'x2': x2,
                    'y2': y2,
                    'confidence': confidence,
                    'cls_id': cls_id,
                    'cls_name': cls_name,
                    'model': model_name,
                })
        
        
        fr = frame.copy()

        for d in results:
            color = get_box_color(d['model'], d['cls_name'])

            name = f'{d["model"]}_{d["cls_name"]}, {d["confidence"]:.2f}'

            x1, y1, x2, y2 = d['x1'], d['y1'], d['x2'], d['y2']
            cv2.rectangle(fr, (x1, y1), (x2, y2), color, 2)
            cv2.putText(fr, name, (x1, y1),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(fr)
        cv2.imshow('proceed_frame', fr)
    out.release()
    cap.release()
    cv2.destroyAllWindows()

In [13]:
out.release()
cap.release()
cv2.destroyAllWindows()

In [14]:
cv2.destroyAllWindows()